In [1]:
import pandas as pd
import numpy as np

In [2]:
def filetodf(filename,time_list=[]):
    # функция считывает файлы из директории и складывает данные в датафрейм
    #  входные данные: 
    #     filename - путь к файлам
    #     time_list - список полей с датой
    #  выходные данные: 
    #     pd.dataframe
    
    import glob
    import pandas as pd
    
    AllFiles = glob.glob(filename)
   
    list_ = [pd.DataFrame()]

    for file_ in AllFiles:
        columns=pd.read_csv(file_, sep=',', nrows=0).columns.str.lower().str.strip()
        df = pd.read_csv(file_,',',names=columns,parse_dates=time_list,dayfirst=True,skiprows=1)
        list_.append(df)
    
    return pd.concat(list_, axis = 0, ignore_index = True, sort=True)

In [3]:
df_sysMetrics = filetodf('input/GrpStat_OSGLOB*.dat')
df_grpMetrics = filetodf('input/GrpStat_RGALL*.dat')
df_directives = filetodf('input/GrpStat_Directives*.dat',time_list=['begin_time'])

df_sysMetrics.drop(['time'], axis=1,inplace=True)
df_grpMetrics.drop(['t_beg'], axis=1,inplace=True)

df_grpMetrics.rename(index=str, columns={"instance_number": "host", "usr_group": "consumer_group"}, inplace=True)
df_directives.rename(index=str, columns={"instance_number": "host", "begin_time": "time"}, inplace=True)

df_grpMetrics['consumer_group']=df_grpMetrics['consumer_group'].str.strip().str.lower()
df_directives['consumer_group']=df_directives['consumer_group'].str.strip().str.lower()
df_directives['plan_name']=df_directives['plan_name'].str.strip().str.lower()

df_grpMetrics.replace(' None', np.NaN, inplace=True)
df_directives.replace(' None', np.NaN, inplace=True)
df_sysMetrics.replace(' None', np.NaN, inplace=True)

df_directives.parallel_target_percentage=df_directives.parallel_target_percentage.astype(float)
df_directives.parallel_degree_limit_p1=df_directives.parallel_degree_limit_p1.astype(float)
df_directives.max_utilization_limit=df_directives.max_utilization_limit.astype(float)
df_grpMetrics.pga=df_grpMetrics.pga.astype(float)

In [4]:
#джойним все данные до уровня host, snap_id, consumer_group
df=df_grpMetrics.set_index(['host','snap_id','consumer_group']) \
.join(df_directives.set_index(['host','snap_id','consumer_group']))\
.reset_index()\
.merge(df_sysMetrics,on=['host','snap_id'])


In [ ]:
#df = df[df.consumer_group!='other_groups']
#df = df[df.consumer_group!='ods2exa_group']

In [5]:
# ресэмплинг до часа, чтобы сгладить ряд
# заполняем отсутствующие значения средними
# фильтруем неполные дни в начале и в конце

df = df.sort_values(['host','consumer_group','time'])
df = df.set_index(pd.DatetimeIndex(df['time']))
#-----------------------------
#df.loc[(df.consumer_group=='adw_unload_group')&(df.plan_name=='sandbox_plan'),'mgmt_p1']=26.
#-----------------------------

#df.max_utilization_limit.fillna(100,inplace=True)
#df.parallel_target_percentage.fillna(100,inplace=True)
#df.loc[:,'cpu_pct_maxul']=df.cpu/df.max_utilization_limit
#df.loc[:,'px_pct_parallel']=df.pxqry/(df.parallel_target_percentage*1400.0/100.0)
#df.loc[df.px_pct_parallel>2,'px_pct_parallel']=2.0
#df.loc[df.cpu_pct_maxul>2,'cpu_pct_maxul']=2.0
#df['CPU_min_treshold']=(df.mgmt_p1/df.max_utilization_limit)

columns=df.columns.drop(['host','consumer_group','time','snap_id','plan_name'],errors='ignore')

df=df.groupby(['host','consumer_group'])[columns].apply(lambda df: df.resample('H').mean().fillna(df.mean(axis=0))).reset_index(['host','consumer_group'])
df=df.groupby(['host','consumer_group',df.index.date]).filter(lambda x: (x.index.min().hour==0)&(x.index.max().hour==23)).reset_index()


In [ ]:
#df_week_shift=df[columns].shift(24*7)
#df_day_shift=df[columns].shift(24)
#for (host,group) in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values:
#    t=df[(df.host==host)&(df.consumer_group==group)][columns]
#    for row in t.index:
#        if t.loc[row].isnull().all():
#            row_day=df_day_shift.loc[row,columns]
#            row_week=df_week_shift.loc[row,columns]
#            if row_week.isnull().all():
#                if row_day.isnull().all():
#                    df.loc[row,columns]=t.loc[:,columns].mean(axis=0)
#                else:
#                    df.loc[row,columns]=row_day
#            else:
#                df.loc[row,columns]=row_week
#df.fillna(0,inplace=True)
            
            

In [6]:
df.to_csv('clear_data/rm_features.csv',';',index=False)